In [ ]:
##################################
# By: Robert Morris
##################################

import numpy as np
import pandas as pd
import os
import glob
import gseapy

def oDir(d):
    isExist = os.path.exists(d)
    if not isExist:
        os.makedirs(d)
        return(d)
    else:
        return(d)

topDir = oDir(d="/output/to/GSEApy_proteinClusterDE1")
outDir = oDir(d=topDir + '/' + 'result')
resDir = "/input/from/differentialExpression/result/directory";

In [ ]:
#read files of DE results
de_files = glob.glob(resDir + "/*_comparison.txt")
gsea_dbids1 = ["/MSigDB/HumanSigDB/c2.cp.v2023.1.Hs.symbols.gmt",
"/MSigDB/HumanSigDB/c5.go.bp.v2023.1.Hs.symbols.gmt",
"/MSigDB/HumanSigDB/h.all.v2023.1.Hs.symbols.gmt"]
gsea_dbids2 = ["/Enrichr/PPI_Hub_Poteins.gmt"]

import re
def GetAllMatches(l1, pattern):
    rs=[]
    cnt = 0
    for m in l1:
        res = re.search(pattern, m)
        rs.append(res.groups())
    r_df = pd.DataFrame(rs)
    r_df['String']=l1
    #print(r_df.shape)
    #print(r_df)
    return(r_df)

#get differential expression results
de_strm = GetAllMatches(l1=de_files, pattern=r'.*/result/(.*)_ProteinL2omMAD_comparison.txt')
#get the signature collection files
db_strm1 = GetAllMatches(l1=gsea_dbids1, pattern=r'.*/HumanSigDB/(.*).Hs.symbols.gmt')
db_strm2 = GetAllMatches(l1=gsea_dbids2, pattern=r'.*/Enrichr/(.*).gmt')
db_strm = db_strm1 + db_strm2


In [ ]:
backgrd_genes=pd.read_csv('QuantifiedProteinsGeneNames.txt', sep='\t', header=None)
backgrd_genes = backgrd_genes[0].squeeze().str.strip().to_list()


In [ ]:

for de_ind in de_strm.index:
    #print(file)
    de1 = pd.read_csv(de_strm['String'][de_ind], sep='\t')
    print('flag1')
    #print(de1.info())
    de1f = de1[(de1['adj.P.Val'] < 0.1) & (de1['logFC'] > 0)]
    de1f.to_csv(outDir + "/" + de_strm[0][de_ind] + "_FDR"+str(0.1)+"_upReg_diffExpr_table.tsv", sep='\t')
    #print(de1f.info())
    #print(nrow(de1f))
    de_up1 = de1f['GeneName'].squeeze().str.strip().to_list()
    print(len(de_up1))
    for db_ind in db_strm.index:
        enr1 = gseapy.enrich(gene_list=de_up1, gene_sets=db_strm['String'][db_ind], outdir=None, background=backgrd_genes, verbose=True)
        enr1_res = enr1.results
        print(type(enr1_res))
        enr1_res.to_csv(outDir + "/" + de_strm[0][de_ind] + "_FDR"+str(0.1)+"_upReg"+"__" + db_strm[0][db_ind] + "gseapy_results.tsv", sep='\t')
    #loop for downregulate proteins
    de2f = de1[(de1['adj.P.Val'] < 0.1) & (de1['logFC'] < 0)]
    de2f.to_csv(outDir + "/" + de_strm[0][de_ind] + "_FDR"+str(0.1)+"_dwnReg_diffExpr_table.tsv", sep='\t')
    #print(de1f.info())
    #print(nrow(de1f))
    de_dwn1 = de2f['GeneName'].squeeze().str.strip().to_list()
    print(len(de_dwn1))
    for db_ind in db_strm.index:
        enr1 = gseapy.enrich(gene_list=de_dwn1, gene_sets=db_strm['String'][db_ind], outdir=None, background=backgrd_genes, verbose=True)
        enr1_res = enr1.results
        print(type(enr1_res))
        enr1_res.to_csv(outDir + "/" + de_strm[0][de_ind] + "_FDR"+str(0.1)+"_dwnReg"+"__" + db_strm[0][db_ind] + "gseapy_results.tsv", sep='\t')

    
    
    
    